<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/zerlina-compute-physical/multi_align.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/zerlina-compute-physical/compute_physical.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

<a href="https://github.com/WinetraubLab/coregister-xy/blob/zerlina-compute-physical/compute_physical.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a>

# Overview
Use this notebook to get information from ImageJ image registration.

In [2]:
# @title Notebook Inputs { display-mode: "form" }
# @markdown How to use this notebook: [See Instructions](https://docs.google.com/document/d/1nQlfuT4SuS7ZfISCR6euZAsNqqRRvODBTRkTy_pmWWs/edit?usp=sharing)

# @markdown Input Paths:
# @markdown Leave either path blank to load a file from local file system.
trakem_xml_path = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/alignment.xml" # @param {type:"string"}
fluorescent_patch_number = 8 # @param {type:"integer"}
# @markdown For the alignment of multiple templates to one fluorescent image, specify the patch numbers of each template in the TrakEM stack:
template_patch_1 = 11 # @param {type:"integer"}
template_patch_2 = 14 # @param {type:"integer"}

template_size = 401

# @markdown Optional: Paths to XML files with exported landmarks. One file per barcode pair. Export each set of selected landmarks.
landmarks_file_1 = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/landmarks1.xml" # @param {type:"string"}
landmarks_file_2 = "landmarks2.xml/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/landmarks2.xml" # @param {type:"string"}

landmarks_files = [landmarks_file_1, landmarks_file_2]

# import cv2 as cv
# from PIL import Image
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')

if not trakem_xml_path:
  print("Upload saved TrakEM project:")
  uploaded = files.upload()
  trakem_xml_path = list(uploaded.keys())[0]
  trakem_xml_path = os.path.join(os.getcwd(), trakem_xml_path)

In [ ]:
# @title Environment Setup
!pip install transforms3d
!git clone -b zerlina-compute-physical https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

# from plane.fit_plane import FitPlane
from plane.parse_xml import ParseXML
import matplotlib.pyplot as plt
import os
from google.colab import files
import math

%cd ..

fatal: destination path 'coregister-xy' already exists and is not an empty directory.
/content/coregister-xy
/content


In [1]:
# TEMPORARY-- RUN FOR SETUP WITHOUT GITHUB.
!pip install transforms3d
from parse_xml import ParseXML
import matplotlib.pyplot as plt
import os
from google.colab import files
import math

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.5 MB/s eta 0:00:00


In [6]:
# @title Compute Physical Transform Parameters, relative to image origin
i = 1
ps = []
templates = [template_patch_1, template_patch_2]
for i in range(0,2):
  print("Template ", i, " (patch number ", templates[i], "):\n")
  project = ParseXML.extract_data(trakem_xml_path, fluorescent_patch_number, templates[i], landmarks_files[i], True)
  ps.append(project)
  tx, ty, theta_deg, sx, sy, shear = project.compute_physical_params()
  print("Translation (x,y): ", "{:.2f}".format(tx),", ", "{:.2f}".format(ty))
  print("Rotation (deg): ", "{:.2f}".format(theta_deg))
  print("Scale x: ", "{:.2f}".format(sx))
  print("Scale y: ", "{:.2f}".format(sy))
  print("Shear: ", "{:.2f}".format(shear))
  print("Alignment Error: ", project.find_transformation_error())
  print()
  i += 1

Template  0  (patch number  11 ):

Translation (x,y):  621.09 ,  630.34
Rotation (deg):  -6.40
Scale x:  0.98
Scale y:  0.95
Shear:  0.01
Alignment Error:  4.038620489329687

Template  1  (patch number  14 ):

Translation (x,y):  906.36 ,  182.38
Rotation (deg):  -8.76
Scale x:  0.89
Scale y:  0.86
Shear:  -0.03
Alignment Error:  5.272608428473057



In [5]:
# @title Calculate Distance and Angle between Barcodes
# Calc centers
center_original = np.array([template_size / 2, template_size / 2, 1])
transformed_center1 = ps[0].transform_points(center_original)
transformed_center2 = ps[1].transform_points(center_original)
# Calc distances
dist = np.linalg.norm(transformed_center1 - transformed_center2)
# Angle
theta_radians = np.arctan2(transformed_center1[0] - transformed_center2[0], transformed_center1[1] - transformed_center2[1])
theta_deg = np.degrees(theta_radians)

print("Distance: ", "{:.2f}".format(dist))
print("Angle (deg): ", "{:.2f}".format(theta_deg))

Distance:  540.06
Angle (deg):  -29.37
